The below script has GUI interface but without Chunks and Delay to reduced TTS API Error 

In [2]:
import PyPDF2
import pyttsx3
import tkinter as tk
from tkinter import *
from tkinter import Label, Button, filedialog, ttk
from tkinter import messagebox as msgb
import subprocess
from openai import OpenAI
from gtts import gTTS
import os
import shutil
import time
import re

In [3]:
# Alternative TTS libraries for more reliability
try:
    from gtts import gTTS
except ImportError:
    gTTS = None

try:
    import pyttsx3
except ImportError:
    pyttsx3 = None

class AudiobookConverter:
    def __init__(self):
        # UI Constants
        self.TITLE_FONT = "Lato"
        self.TITLE_COLOR = "#FFA500"
        self.WORD_FONT = "Lato"
        self.WORD_COLOR = "#FFB347"
        self.BACKGROUND_COLOR = "#000000"

        # Directories
        self.UPLOAD_DIR = "uploaded_pdf"
        self.AUDIO_DIR = "audios"

        # Setup main window
        self.root = tk.Tk()
        self.root.title("Convert to Audiobook")
        self.root.config(padx=20, pady=20, bg=self.BACKGROUND_COLOR)
        
        # Create necessary directories
        self._create_directories()
        
        # Setup UI
        self._create_ui()

    def _create_directories(self):
        """Create necessary directories for storing files."""
        for directory in [self.UPLOAD_DIR, self.AUDIO_DIR]:
            os.makedirs(directory, exist_ok=True)

    def _create_ui(self):
        """Create and layout UI components."""
        # Title
        title_label = tk.Label(
            self.root, 
            text="Convert to Audiobook", 
            font=(self.TITLE_FONT, 40, "bold"),
            fg=self.TITLE_COLOR, 
            bg=self.BACKGROUND_COLOR
        )
        title_label.pack(pady=10)

        # Upload Button
        self.upload_btn = tk.Button(
            self.root, 
            text="Select PDF File", 
            font=(self.WORD_FONT, 15),
            fg=self.WORD_COLOR, 
            bg=self.BACKGROUND_COLOR, 
            padx=10, 
            pady=5,
            command=self.select_pdf
        )
        self.upload_btn.pack(pady=5)

        # Play Button (initially hidden)
        self.play_btn = tk.Button(
            self.root, 
            text="Play Audiobook", 
            font=(self.WORD_FONT, 15),
            fg=self.WORD_COLOR, 
            bg=self.BACKGROUND_COLOR, 
            padx=10, 
            pady=5,
            command=self.play_audio
        )

        # Download Button (initially hidden)
        self.download_btn = tk.Button(
            self.root, 
            text="Download Audiobook", 
            font=(self.WORD_FONT, 15),
            fg=self.WORD_COLOR, 
            bg=self.BACKGROUND_COLOR, 
            padx=10, 
            pady=5,
            command=self.download_audio
        )

        # Close Button
        self.close_btn = tk.Button(
            self.root, 
            text="Close App", 
            font=(self.WORD_FONT, 15),
            fg=self.WORD_COLOR, 
            bg=self.BACKGROUND_COLOR, 
            padx=10, 
            pady=5,
            command=self.close_app
        )
        self.close_btn.pack(pady=5)

    def _text_to_speech_gtts(self, text, audio_path):
        """Generate speech using Google Text-to-Speech."""
        if gTTS is None:
            raise ImportError("gTTS is not installed. Please install it using 'pip install gtts'")
        
        try:
            tts = gTTS(text)
            tts.save(audio_path)
            return True
        except Exception as e:
            print(f"gTTS Error: {e}")
            return False

    def _text_to_speech_pyttsx3(self, text, audio_path):
        """Generate speech using pyttsx3 (local TTS engine)."""
        if pyttsx3 is None:
            raise ImportError("pyttsx3 is not installed. Please install it using 'pip install pyttsx3'")
        
        try:
            engine = pyttsx3.init()
            engine.save_to_file(text, audio_path)
            engine.runAndWait()
            return True
        except Exception as e:
            print(f"pyttsx3 Error: {e}")
            return False

    def select_pdf(self):
        """Select and process PDF file."""
        try:
            # Open file dialog
            filename = filedialog.askopenfilename(
                initialdir="/Downloads", 
                title="Select PDF File",
                filetypes=[("PDF files", "*.pdf"), ("All files", "*.*")]
            )
            
            # Validate file selection
            if not filename:
                messagebox.showerror("Error", "No file was chosen. Please choose a file.")
                return

            # Copy PDF to upload directory
            pdf_path = os.path.join(self.UPLOAD_DIR, "file.pdf")
            shutil.copy(filename, pdf_path)

            # Extract text from PDF
            with open(pdf_path, "rb") as pdf_file:
                pdfreader = PyPDF2.PdfReader(pdf_file)
                all_text = " ".join(
                    page.extract_text().strip().replace("\n", " ")
                    for page in pdfreader.pages
                )

            # Validate text extraction
            if not all_text.strip():
                messagebox.showerror("Error", "The selected PDF is empty or cannot be processed.")
                return

            # Generate audio
            audio_path = os.path.join(self.AUDIO_DIR, "audio.mp3")
            
            # Try multiple TTS methods
            tts_success = False
            
            # Try gTTS first
            if gTTS is not None:
                tts_success = self._text_to_speech_gtts(all_text, audio_path)
            
            # If gTTS fails, try pyttsx3
            if not tts_success and pyttsx3 is not None:
                tts_success = self._text_to_speech_pyttsx3(all_text, audio_path)
            
            # Check if TTS conversion was successful
            if not tts_success:
                messagebox.showerror("Error", "Failed to convert text to speech. Try installing gTTS or pyttsx3.")
                return

            # Show additional buttons
            self.play_btn.pack(pady=5)
            self.download_btn.pack(pady=5)

            messagebox.showinfo("Success", "Audiobook created successfully!")

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def play_audio(self):
        """Play the generated audiobook."""
        audio_file_path = os.path.join(self.AUDIO_DIR, "audio.mp3")
        if os.path.exists(audio_file_path):
            # Use platform-independent method to open audio
            if os.name == 'nt':  # Windows
                os.startfile(audio_file_path)
            elif os.name == 'posix':  # macOS and Linux
                subprocess.call((['open', audio_file_path]) if os.sys.platform == 'darwin' else ['xdg-open', audio_file_path])
        else:
            messagebox.showerror("Error", "No audio file found. Please create an audiobook first.")

    def download_audio(self):
        """Download the generated audiobook."""
        audio_path = os.path.join(self.AUDIO_DIR, "audio.mp3")
        if not os.path.exists(audio_path):
            messagebox.showerror("Error", "No audio file found to download.")
            return

        # Open save dialog
        download_path = filedialog.asksaveasfilename(
            initialdir="/", 
            title="Save Audio As",
            defaultextension=".mp3",
            filetypes=[("MP3 files", "*.mp3"), ("All files", "*.*")]
        )

        if not download_path:
            return

        try:
            shutil.copy(audio_path, download_path)
            messagebox.showinfo("Success", "Audio downloaded successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to download audio: {e}")

    def close_app(self):
        """Close the application after confirmation."""
        if messagebox.askokcancel("Quit", "Do you want to close the app? Make sure that you downloaded the audio."):
            self.root.destroy()

    def run(self):
        """Start the Tkinter event loop."""
        self.root.mainloop()

def main():
    """Main entry point of the application."""
    converter = AudiobookConverter()
    converter.run()

if __name__ == "__main__":
    main()

In [ ]:
echo "# Audiobook" >> README. md
git init
git add README. md
git commit -m "first commit"
git remote add origin https: //github.com/Ojchavan/Audiobook.git
git push -u origin master